In [1]:
import pandas as pd
import glob
from datetime import datetime, timedelta

from openpyxl.reader.excel import load_workbook
from openpyxl.styles import Font
from openpyxl.styles.fills import PatternFill
from openpyxl.styles import Border, Side

In [2]:
files = [item for item in glob.glob(r'/Users/iaroslavbezukladnikov/Desktop/excel_file/*{}'.format('.xlsx'))]


In [3]:
combined = pd.DataFrame()
for file in files:
    file = pd.read_excel(file, 
                         skiprows = 5, usecols=['ОМСУ','Наименование УК','Номер Добродела','№ ЕДС','Дата обращения', 
                                                'Адрес заявки','Срок исполнения', 'Текущий статус заявки',
                                                'Текущий статус заявки',
                                                'Текст заявки', 'Ответ УК предоставлен?'])
    file = file [['ОМСУ', 'Наименование УК','Номер Добродела','№ ЕДС','Дата обращения', 
                                                'Адрес заявки','Срок исполнения', 'Срок исполнения', 'Текущий статус заявки',
                                                  'Текущий статус заявки', 
                                                'Текст заявки', 'Ответ УК предоставлен?']]
    
    file.columns = ['ОМСУ', 'Наименование УК','Номер Добродела','№ ЕДС','Дата обращения', 
                                                'Адрес заявки','Срок исполнения', 'temp_date', 'Текущий статус заявки',
                                                  'k', 
                                                'Текст заявки', 'Ответ УК предоставлен?']
    combined = pd.concat([combined, file])
  

/Users/iaroslavbezukladnikov/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/iaroslavbezukladnikov/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/iaroslavbezukladnikov/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/Users/iaroslavbezukladnikov/opt/anaconda3/lib/python3.9/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's defaul

In [4]:
status_in_table=[]
status_need = ['В работе', 'Контроль ГЖИ: Просрочено', 'Новая заявка', 'Срок превышен', 
               'Требуется доработка', 'Требуется доработка (c просрочкой)','Контроль ГЖИ: Устранение']
for i in combined['Текущий статус заявки']:
    if i not in status_need and i not in status_in_table: 
        status_in_table.append(i)

**Работаем со столбцом индекса. Вместо порядкого номера по умолчанию меняем его на столбец Текущий статус заявки. 

In [5]:
combined = combined.set_index('k')



In [6]:
combined.drop(status_in_table, axis=0,inplace = True)


# Начнем работу по сортировки таблице по дате. 

In [7]:
combined['temp_date'] = pd.to_datetime(combined['temp_date'],  format='%d.%m.%Y %H:%M') # Преобразовали тип данных даты к нужному формату

In [8]:
combined = combined.set_index('temp_date')

In [9]:
combined = combined.sort_index(ascending=True) #сотировка по столбцу индекса по возрастанию. 


# Устанавливаю текущую дату, прибавляю к ней 1 день и преобразую в строку только дату. 

In [10]:
if datetime.weekday(datetime.now()) == 4:
    current_date = (datetime.now() + timedelta(days=2)).strftime("%Y-%m-%d")
else:
    current_date = (datetime.now() + timedelta(days=1)).strftime("%Y-%m-%d")

In [11]:
combined = combined.loc[:current_date] # Делаю срез по столбцу индекса до текущей дату плюс один день 

# Закрасим ячейки. 

 **Делаем экспорт файла со сбросом столбца индекса. index= False.


In [12]:
combined.to_excel(r'/Users/iaroslavbezukladnikov/Desktop/excel_file/res1.xlsx', index= False)

# Форматируем итоговый отчет. 

In [13]:
book = load_workbook('/Users/iaroslavbezukladnikov/Desktop/excel_file/res1.xlsx')
ws = book.worksheets[0]
ws.column_dimensions["A"].width = 20
ws.column_dimensions["B"].width = 30
ws.column_dimensions["C"].width = 8
ws.column_dimensions["D"].width = 23
ws.column_dimensions["E"].width = 16
ws.column_dimensions["F"].width = 25
ws.column_dimensions["G"].width = 16
ws.column_dimensions["H"].width = 22
ws.column_dimensions["I"].width = 40
ws.column_dimensions["J"].width = 40



# Задаем стиль для ячейки
count = 2

while True:
    
    _cell = ws['G' + str(count)]
    now_date = datetime.now().date()
    try:
        date_time_obj = datetime.strptime(_cell.value, '%d.%m.%Y %H:%M')
        
    except TypeError:
        break
        
    if date_time_obj.date() < now_date:
        _cell.fill = PatternFill("solid", fgColor="00FF0000")
        
    elif date_time_obj.date() == now_date:
        _cell.fill = PatternFill("solid", fgColor="00FFCC99")
        
    else:
        _cell.fill = PatternFill("solid", fgColor="00FFFF00")
        
    count += 1

book.save('/Users/iaroslavbezukladnikov/Desktop/excel_file/res1.xlsx')